In [1]:
!pip install pandas


Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install duckdb


Defaulting to user installation because normal site-packages is not writeable


## Exploratory data analysis (EDA)

In [2]:
import pandas as pd
import duckdb

df = pd.read_csv('ecommerce_sales_large.csv')  # Load your CSV
df


,order_id,customer_id,product_id,product_category,order_date,quantity,unit_price,total_price,region
0,1,CUST015796,PROD0111,Books,15-02-2023,4,642.34,2569.36,South
1,2,CUST000861,PROD0142,Home,02-12-2023,1,337.05,337.05,South
2,3,CUST076821,PROD0007,Clothing,27-09-2024,2,763.14,1526.28,East
3,4,CUST054887,PROD0077,Books,20-01-2023,1,194.33,194.33,South
4,5,CUST006266,PROD0001,Clothing,10-02-2024,4,458.94,1835.76,South
...,...,...,...,...,...,...,...,...,...
999995,999996,CUST066770,PROD0013,Home,06-03-2023,3,285.79,857.37,South
999996,999997,CUST053113,PROD0197,Home,21-07-2023,2,690.25,1380.50,North
999997,999998,CUST054332,PROD0023,Clothing,16-07-2024,1,442.03,442.03,North
999998,999999,CUST051995,PROD0044,Electronics,19-02-2024,4,616.58,2466.32,South


##### Total rows count

In [3]:
duckdb.query("SELECT COUNT(*) as total_rows FROM df").show()  # Run SQL directly on the DataFrame

┌────────────┐
│ total_rows │
│   int64    │
├────────────┤
│    1000000 │
└────────────┘



##### Data Type of columns

In [40]:
duckdb.query("DESCRIBE df").to_df()


,column_name,column_type,null,key,default,extra
0,order_id,BIGINT,YES,None,None,None
1,customer_id,VARCHAR,YES,None,None,None
2,product_id,VARCHAR,YES,None,None,None
3,product_category,VARCHAR,YES,None,None,None
4,order_date,VARCHAR,YES,None,None,None
5,quantity,BIGINT,YES,None,None,None
6,unit_price,DOUBLE,YES,None,None,None
7,total_price,DOUBLE,YES,None,None,None
8,region,VARCHAR,YES,None,None,None


##### Changing the data type of the order_date column from VARCHAR to TIMESTAMP

In [7]:
ecommerce_sales = duckdb.query("""
    SELECT 
        order_id,
        customer_id,
        product_id,
        product_category,
        STRPTIME(order_date, '%d-%m-%Y') AS order_date,  
        quantity,
        unit_price,
        total_price,
        region
    FROM df
""").to_df()


In [8]:
duckdb.query("DESCRIBE ecommerce_sales").to_df()


,column_name,column_type,null,key,default,extra
0,order_id,BIGINT,YES,None,None,None
1,customer_id,VARCHAR,YES,None,None,None
2,product_id,VARCHAR,YES,None,None,None
3,product_category,VARCHAR,YES,None,None,None
4,order_date,TIMESTAMP,YES,None,None,None
5,quantity,BIGINT,YES,None,None,None
6,unit_price,DOUBLE,YES,None,None,None
7,total_price,DOUBLE,YES,None,None,None
8,region,VARCHAR,YES,None,None,None


##### Check for missing values

In [9]:
duckdb.query("""SELECT 
    SUM(CASE WHEN order_id IS NULL THEN 1 ELSE 0 END) AS missing_order_id,
    SUM(CASE WHEN customer_id IS NULL THEN 1 ELSE 0 END) AS missing_customer_id,
    SUM(CASE WHEN product_id IS NULL THEN 1 ELSE 0 END) AS missing_product_id,
    SUM(CASE WHEN product_category IS NULL THEN 1 ELSE 0 END) AS missing_product_category,
    SUM(CASE WHEN order_date IS NULL THEN 1 ELSE 0 END) AS missing_order_date,
    SUM(CASE WHEN quantity IS NULL THEN 1 ELSE 0 END) AS missing_quantity,
    SUM(CASE WHEN unit_price IS NULL THEN 1 ELSE 0 END) AS missing_unit_price,
    SUM(CASE WHEN total_price IS NULL THEN 1 ELSE 0 END) AS missing_total_price,
    SUM(CASE WHEN region IS NULL THEN 1 ELSE 0 END) AS missing_region
FROM ecommerce_sales""" ).to_df()


,missing_order_id,missing_customer_id,missing_product_id,missing_product_category,missing_order_date,missing_quantity,missing_unit_price,missing_total_price,missing_region
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##### Unique value counts

In [12]:
duckdb.query("""SELECT 
    COUNT(DISTINCT customer_id) AS unique_customers,
    COUNT(DISTINCT order_id) AS unique_orders,
    COUNT(DISTINCT product_id) AS unique_products,
    COUNT(DISTINCT product_category) AS unique_categories,
    COUNT(DISTINCT region) AS unique_regions
FROM ecommerce_sales""" ).to_df()


,unique_customers,unique_orders,unique_products,unique_categories,unique_regions
0,99990,1000000,200,4,4


##### Top-selling products

In [14]:
duckdb.query("""SELECT product_id, SUM(quantity) AS total_sold
FROM ecommerce_sales
GROUP BY product_id
ORDER BY total_sold DESC
LIMIT 10""" ).to_df()


,product_id,total_sold
0,PROD0018,15708.0
1,PROD0073,15648.0
2,PROD0081,15544.0
3,PROD0179,15520.0
4,PROD0088,15456.0
5,PROD0014,15435.0
6,PROD0071,15429.0
7,PROD0194,15425.0
8,PROD0075,15419.0
9,PROD0094,15404.0


#####  Revenue by product category

In [17]:
duckdb.query("""SELECT product_category, SUM(total_price) AS total_revenue
FROM ecommerce_sales
GROUP BY product_category
ORDER BY total_revenue DESC""" )


┌──────────────────┬────────────────────┐
│ product_category │   total_revenue    │
│     varchar      │       double       │
├──────────────────┼────────────────────┤
│ Clothing         │ 365271155.64000034 │
│ Electronics      │ 365057355.26000124 │
│ Books            │ 242845179.99000046 │
│ Home             │ 242106718.10999957 │
└──────────────────┴────────────────────┘

##### Monthly sales trend

In [20]:
duckdb.query("""SELECT 
    strftime(order_date, '%Y-%m') AS month_year,
    SUM(total_price) AS monthly_sales
FROM ecommerce_sales
GROUP BY month_year
ORDER BY month_year;
""" )


┌────────────┬────────────────────┐
│ month_year │   monthly_sales    │
│  varchar   │       double       │
├────────────┼────────────────────┤
│ 2023-01    │  51296522.58000007 │
│ 2023-02    │  46847071.96000007 │
│ 2023-03    │ 51501583.849999845 │
│ 2023-04    │  50173870.85999997 │
│ 2023-05    │         51632468.8 │
│ 2023-06    │ 49744923.480000004 │
│ 2023-07    │ 51508014.689999945 │
│ 2023-08    │  51083366.11000001 │
│ 2023-09    │  50151097.49999998 │
│ 2023-10    │  51231497.86000001 │
│    ·       │          ·         │
│    ·       │          ·         │
│    ·       │          ·         │
│ 2024-03    │  51711954.12000001 │
│ 2024-04    │ 50468075.889999986 │
│ 2024-05    │  51465499.53999999 │
│ 2024-06    │  49262056.93999999 │
│ 2024-07    │  51164629.07999997 │
│ 2024-08    │ 51736610.050000004 │
│ 2024-09    │ 50204958.010000095 │
│ 2024-10    │  51775196.34000009 │
│ 2024-11    │        49586355.28 │
│ 2024-12    │  51534992.99000005 │
├────────────┴──────────────

##### Sales by region

In [23]:
duckdb.query("""SELECT region, SUM(total_price) AS region_sales
FROM ecommerce_sales
GROUP BY region
ORDER BY region_sales DESC""" )


┌─────────┬────────────────────┐
│ region  │    region_sales    │
│ varchar │       double       │
├─────────┼────────────────────┤
│ South   │  364139071.4000005 │
│ North   │ 364027376.69000006 │
│ West    │ 243915389.40999973 │
│ East    │        243198571.5 │
└─────────┴────────────────────┘

#### Repeat vs One-time Customers

In [24]:
duckdb.query("""SELECT
    CASE 
        WHEN order_count = 1 THEN 'One-time'
        ELSE 'Repeat'
    END AS customer_type,
    COUNT(*) AS customer_count
FROM (
    SELECT customer_id, COUNT(DISTINCT order_id) AS order_count
    FROM ecommerce_sales
    GROUP BY customer_id
)
GROUP BY customer_type;""" )


┌───────────────┬────────────────┐
│ customer_type │ customer_count │
│    varchar    │     int64      │
├───────────────┼────────────────┤
│ Repeat        │          99944 │
│ One-time      │             46 │
└───────────────┴────────────────┘

## Solution for Assignment Questions

### 1)  Customer Repeat Purchases

##### 1st approach

In [7]:
Customer_Repeat_Purchases = duckdb.query("""
    SELECT 
        a.customer_id,
        a.order_id AS first_order_id,
        b.order_id AS second_order_id,
        DATE_DIFF('day', a.order_date, b.order_date) AS day_difference
    FROM ecommerce_sales a
    JOIN ecommerce_sales b
        ON a.customer_id = b.customer_id
    where b.order_date > a.order_date
       AND DATE_DIFF('day', a.order_date, b.order_date) <= 30
    ORDER BY a.customer_id, a.order_date
""").to_df()

Customer_Repeat_Purchases


,customer_id,first_order_id,second_order_id,day_difference
0,CUST000001,726450,171326,24
1,CUST000001,171326,355181,12
2,CUST000002,376505,402748,13
3,CUST000003,10530,48954,22
4,CUST000003,625678,62161,3
...,...,...,...,...
402662,CUST099999,668918,578181,2
402663,CUST099999,668918,708262,22
402664,CUST099999,578181,708262,20
402665,CUST099999,708262,723962,14


##### 2nd approach

In [43]:
Customer_Repeat_Purchases_2 = duckdb.query(""" 
SELECT *
FROM (
    SELECT
        customer_id,
        order_id AS first_order_id,
        LEAD(order_id) OVER (PARTITION BY customer_id ORDER BY order_date) AS second_order_id,
        DATE_DIFF('day', order_date, LEAD(order_date) OVER (PARTITION BY customer_id ORDER BY order_date)) AS day_difference
    FROM ecommerce_sales
) t
WHERE day_difference <= 30 
order by customer_id """ ).to_df()

Customer_Repeat_Purchases_2


,customer_id,first_order_id,second_order_id,day_difference
0,CUST000001,726450,171326,24
1,CUST000001,171326,355181,12
2,CUST000002,376505,402748,13
3,CUST000003,10530,48954,22
4,CUST000003,625678,62161,3
...,...,...,...,...
334800,CUST099999,539372,668918,3
334801,CUST099999,668918,578181,2
334802,CUST099999,578181,708262,20
334803,CUST099999,708262,723962,14


### 2)   Products with Similar Prices

In [45]:
Products_with_Similar_Prices = duckdb.query("""
WITH ordered_products AS (
  SELECT 
    product_id,
    product_category,
    unit_price,
    LEAD(product_id) OVER (PARTITION BY product_category ORDER BY unit_price) AS next_product,
    LEAD(unit_price) OVER (PARTITION BY product_category ORDER BY unit_price) AS next_price
  FROM ecommerce_sales
)
SELECT 
  product_category,
  product_id AS product_id_1,
  next_product AS product_id_2,
  unit_price AS unit_price_1,
  next_price AS unit_price_2,
  ABS(unit_price - next_price) / unit_price AS price_diff_ratio
FROM ordered_products
WHERE ABS(unit_price - next_price) / unit_price < 0.1  
order by product_category""" ).to_df()

Products_with_Similar_Prices


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,product_category,product_id_1,product_id_2,unit_price_1,unit_price_2,price_diff_ratio
0,Books,PROD0121,PROD0059,405.85,405.85,0.000000
1,Books,PROD0092,PROD0082,603.13,603.13,0.000000
2,Books,PROD0123,PROD0057,208.80,208.80,0.000000
3,Books,PROD0057,PROD0081,208.80,208.80,0.000000
4,Books,PROD0110,PROD0167,10.00,10.01,0.001000
...,...,...,...,...,...,...
999991,Home,PROD0062,PROD0186,466.92,466.92,0.000000
999992,Home,PROD0061,PROD0194,466.91,466.91,0.000000
999993,Home,PROD0101,PROD0155,690.21,690.21,0.000000
999994,Home,PROD0065,PROD0014,799.98,799.99,0.000013


### 3)   Top Customers by Region

In [48]:
Top_Customers_by_Region  = duckdb.query(""" 
select * from (
SELECT 
    region,
    customer_id,
    SUM(total_price) AS total_sales,
    DENSE_RANK() OVER (PARTITION BY region ORDER BY SUM(total_price) DESC) AS sales_rank
FROM ecommerce_sales
GROUP BY region, customer_id )
where  sales_rank <= 3 
order by region , sales_rank""").to_df()

Top_Customers_by_Region


,region,customer_id,total_sales,sales_rank
0,East,CUST035845,19363.92,1
1,East,CUST006821,15013.58,2
2,East,CUST001246,14877.96,3
3,North,CUST056304,21457.26,1
4,North,CUST080179,20709.70,2
5,North,CUST002872,20367.17,3
6,South,CUST046721,22784.52,1
7,South,CUST029918,22089.46,2
8,South,CUST079048,20530.55,3
9,West,CUST020006,20815.61,1


In [24]:
Top_Customers_by_Region.to_excel(
    'Top_Customers_by_Region_3.xlsx',
    index=False,
    float_format='%.2f'  # Format floating point numbers to 2 decimal places
)


### 4)  Running Total Sales by Category

##### 1st approach

In [58]:
Running_total_sales_category_2 = duckdb.query(""" 
WITH DailySales AS (
    SELECT
        product_category,
        order_date,
        SUM(total_price) AS daily_total
    FROM
        ecommerce_sales
    GROUP BY
        product_category,
        order_date
)
SELECT
    product_category,
    order_date,
    daily_total,
    SUM(daily_total) OVER (
        PARTITION BY product_category
        ORDER BY order_date
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
    ) AS running_total
FROM
    DailySales
ORDER BY
    product_category,
    order_date """ )

Running_total_sales_category_2



┌──────────────────┬─────────────────────┬────────────────────┬────────────────────┐
│ product_category │     order_date      │    daily_total     │   running_total    │
│     varchar      │      timestamp      │       double       │       double       │
├──────────────────┼─────────────────────┼────────────────────┼────────────────────┤
│ Books            │ 2023-01-01 00:00:00 │          342975.69 │          342975.69 │
│ Books            │ 2023-01-02 00:00:00 │ 345581.20000000007 │  688556.8900000001 │
│ Books            │ 2023-01-03 00:00:00 │          320668.75 │ 1009225.6400000001 │
│ Books            │ 2023-01-04 00:00:00 │           323163.9 │         1332389.54 │
│ Books            │ 2023-01-05 00:00:00 │  327978.8600000001 │ 1660368.4000000001 │
│ Books            │ 2023-01-06 00:00:00 │          323933.59 │ 1984301.9900000002 │
│ Books            │ 2023-01-07 00:00:00 │ 325178.07999999996 │ 2309480.0700000003 │
│ Books            │ 2023-01-08 00:00:00 │ 340768.94000000006 │ 2

##### 2nd approach

In [66]:
Running_total_sales_category_4 = duckdb.query("""
SELECT
    product_category,
    order_date,
    total_price,
    SUM(total_price) OVER (
        PARTITION BY product_category
        ORDER BY order_date, total_price
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
    ) AS running_total
FROM
    ecommerce_sales
ORDER BY
    product_category,
    order_date,
    running_total""" )

Running_total_sales_category_4

┌──────────────────┬─────────────────────┬─────────────┬────────────────────┐
│ product_category │     order_date      │ total_price │   running_total    │
│     varchar      │      timestamp      │   double    │       double       │
├──────────────────┼─────────────────────┼─────────────┼────────────────────┤
│ Books            │ 2023-01-01 00:00:00 │       29.25 │              29.25 │
│ Books            │ 2023-01-01 00:00:00 │       45.22 │              74.47 │
│ Books            │ 2023-01-01 00:00:00 │       75.66 │             150.13 │
│ Books            │ 2023-01-01 00:00:00 │       77.16 │             227.29 │
│ Books            │ 2023-01-01 00:00:00 │       82.31 │              309.6 │
│ Books            │ 2023-01-01 00:00:00 │       83.44 │             393.04 │
│ Books            │ 2023-01-01 00:00:00 │       84.51 │             477.55 │
│ Books            │ 2023-01-01 00:00:00 │        87.0 │             564.55 │
│ Books            │ 2023-01-01 00:00:00 │       88.55 │  653.09